In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

FAST_RUN = False

EPOCHS=1000
BATCH_SIZE = 128
TARGET_SIZE = (180, 180)
THRESHOLD = 0.50
INPUT_PATH = "./kaggle/input/plant-pathology-2021-fgvc8/"
CHECKPOINT_FILEPATH = './tmp/checkpoint'
# SELECT MODEL FILEPATH
#MODEL_FILEPATH = '/kaggle/input/model2/model'
MODEL_FILEPATH = './model'
CLASS_NAMES = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
NUM_CLASSES = len(CLASS_NAMES)

def labels_to_str(vector):
    result = []
    for idx, v in enumerate(vector):
        if v == 1:
            result.append(CLASS_NAMES[idx])
    return " ".join(result)


print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
model = tf.keras.models.load_model(MODEL_FILEPATH)

In [7]:
test_df = pd.read_csv(INPUT_PATH + 'sample_submission.csv')
test_path = INPUT_PATH + 'test_images'

image_test_datagen = ImageDataGenerator(
    rescale=1./255,
)

test_generator = image_test_datagen.flow_from_dataframe(
    test_df,
    directory = test_path,
    x_col = "image",
    y_col = "labels",
    target_size = TARGET_SIZE,
    color_mode="rgb",
    batch_size=1,
    shuffle=False,
    seed=123,
    subset=None
)

Found 3 validated image filenames belonging to 1 classes.


In [8]:
predicts = model.predict(test_generator)
verdict = (predicts>THRESHOLD)
answer = []
for i in range(verdict.shape[0]):
    answer.append(labels_to_str(verdict[i]))


test_df['labels'] = np.array(answer)
print(test_df)
test_df.to_csv('submission.csv', index=False)

3/3 [==============================] - 0s 46ms/step
                  image labels
0  85f8cb619c66b863.jpg   rust
1  ad8770db05586b59.jpg       
2  c7b03e718489f3ca.jpg       
